In [0]:
# BLOB STORAGE INITIALIZATION
try:
    containe_name = "datacontainer"
    storage_account_name = "nithyas3storage"
    accountkey = "Tp8pV6voD3lYkVja/ycqKLSBriDU1gwc0RFgedA7RNH68502aCEY7O4DjjkWC89tlGSbz6twUcFa+AStzNhRmg=="
    config = "fs.azure.account.key.nithyas3storage.blob.core.windows.net"

    dbutils.fs.mount(
        source = "wasbs://{}@{}.blob.core.windows.net/".format(containe_name, storage_account_name),
        mount_point = "/mnt/blobstorage", #blobstorage is mount point name
        extra_configs = {config: accountkey}
    )
    
except Exception as e:
    print(e)
    print("already mounted. Try to unmount first")

In [0]:
confluentClusterName = "cluster_0"
confluentBootstrapServers = "pkc-ymrq7.us-east-2.aws.confluent.cloud:9092"
confluentTopicName = "sales"
schemaRegistryUrl = "https://psrc-4nyjd.us-central1.gcp.confluent.cloud"
confluentApiKey = "4APZABSYMGAGJU5J"
confluentSecret = "S4Mpil9Wnlso8iyyqY+RY4YHUQHjSVyAId+pR4z3vpofTwHrC9ZYsan//l1BCCXg"
confluentRegistryApiKey = "GO5XDKCD65YVGIRD"
confluentRegistrySecret = "iz7yRGcygzQ4ZGSjEC25BLgTFTLcnkotmq03m9KTY2uqqlWRl9YDj3/iqXaUwRTj"



In [0]:
from confluent_kafka.schema_registry import SchemaRegistryClient
import ssl
schema_registry_conf = {
    'url': schemaRegistryUrl,
    'basic.auth.user.info': '{}:{}'.format(confluentRegistryApiKey, confluentRegistrySecret)}
schema_registry_client = SchemaRegistryClient(schema_registry_conf)

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import StructType,StructField, StringType, IntegerType,FloatType,DataType
from pyspark.sql.functions import col,from_json
import pyspark.sql.functions as fn
import pyspark.sql.types as T

In [0]:
df = (
  spark
  .readStream
  .format("kafka")
  .option("kafka.bootstrap.servers", confluentBootstrapServers)
  .option("kafka.security.protocol", "SASL_SSL")
  .option("kafka.sasl.jaas.config", "kafkashaded.org.apache.kafka.common.security.plain.PlainLoginModule required username='{}' password='{}';".format(confluentApiKey, confluentSecret))
  .option("kafka.ssl.endpoint.identification.algorithm", "https")
  .option("kafka.sasl.mechanism", "PLAIN")
  .option("subscribe", confluentTopicName)
  .option("startingOffsets", "earliest")
  .option("failOnDataLoss", "false")
  .option("includeHeaders", "true")
  .load()
)

In [0]:
df = df.selectExpr("CAST(topic AS STRING)", "CAST(partition AS STRING)", "CAST(offset AS STRING)", "CAST(value AS STRING)")

schema = StructType([ 
    StructField("SalesOrderID",IntegerType(),True), 
    StructField("SalesOrderDetailID",IntegerType(),True), 
    StructField("CarrierTrackingNumber",StringType(),True), 
    StructField("OrderQty", IntegerType(), True),
    StructField("ProductID", IntegerType(), True),
    StructField("SpecialOfferID", IntegerType(), True),
    StructField("UnitPrice", FloatType(), True),
    StructField("UnitPriceDiscount", FloatType(), True),
    StructField("LineTotal", FloatType(), True),
    StructField("rowguid", StringType(), True),
    StructField("ModifiedDate", StringType(), True)]) 

In [0]:
df1 = df.withColumn('VALUES', fn.expr("substring(value, 6, length(value)-5)"))
df2 = df1.select("VALUES")
dfJSON = df2.withColumn("jsonData",from_json(col("VALUES"),schema))
DF = dfJSON.select("jsonData.*")
DF.display()

SalesOrderID,SalesOrderDetailID,CarrierTrackingNumber,OrderQty,ProductID,SpecialOfferID,UnitPrice,UnitPriceDiscount,LineTotal,rowguid,ModifiedDate
43659,5,4911-403C-98,1,772,1,2039.994,0.0,2039.994,5A74C7D2-E641-438E-A7AC-37BF23280301,2011-05-30 18:30:00
43659,7,4911-403C-98,1,774,1,2039.994,0.0,2039.994,80667840-F962-4EE3-96E0-AECA108E0D4F,2011-05-30 18:30:00
43659,10,4911-403C-98,6,709,1,5.7,0.0,34.2,AC769034-3C2F-495C-A5A7-3B71CDB25D4E,2011-05-30 18:30:00
43660,14,6431-4D57-83,1,758,1,874.794,0.0,874.794,5D0B2B03-1D4C-4C34-9696-C14C58E7301C,2011-05-30 18:30:00
43661,16,4E0A-4F89-AE,1,743,1,714.7043,0.0,714.7043,FE10BF09-D477-485B-9541-27AE8053A6D4,2011-05-30 18:30:00
43661,24,4E0A-4F89-AE,2,741,1,818.7,0.0,1637.4,2F911593-E3B8-4ADF-B757-7242004FFB8E,2011-05-30 18:30:00
43662,34,2E53-4802-85,3,725,1,183.9382,0.0,551.8146,92897EE9-72A6-49E6-A421-C243B531AFF2,2011-05-30 18:30:00
43662,48,2E53-4802-85,1,733,1,356.898,0.0,356.898,7BA3862C-3C4F-4E87-8C47-25B411FA21E6,2011-05-30 18:30:00
43664,53,2F44-4BA1-BB,1,772,1,2039.994,0.0,2039.994,22DB5FA8-8C63-4A68-A038-435EFD6D7EA2,2011-05-30 18:30:00
43664,55,2F44-4BA1-BB,1,714,1,28.8404,0.0,28.8404,076636AF-A24F-40D7-AEDC-E1564BB16F0E,2011-05-30 18:30:00


In [0]:
DF.coalesce(1).writeStream \ 
.format("csv") \
.option("header",True) \ 
.outputMode("append") \
.option("mergeSchema", "true") \
.option("checkpointLocation", "/mnt/blobstorage/") \
.start("/mnt/blobstorage/") \
.awaitTermination()

java.io.IOException: Connection failed
	at com.databricks.rpc.Jetty9Client$$anon$1.handleError(Jetty9Client.scala:771)
	at com.databricks.rpc.Jetty9Client$$anon$1.onFailure(Jetty9Client.scala:698)
	at shaded.v9_4.org.eclipse.jetty.client.ResponseNotifier.notifyFailure(ResponseNotifier.java:197)
	at shaded.v9_4.org.eclipse.jetty.client.ResponseNotifier.notifyFailure(ResponseNotifier.java:189)
	at shaded.v9_4.org.eclipse.jetty.client.HttpExchange.notifyFailureComplete(HttpExchange.java:268)
	at shaded.v9_4.org.eclipse.jetty.client.HttpExchange.abort(HttpExchange.java:240)
	at shaded.v9_4.org.eclipse.jetty.client.HttpConversation.abort(HttpConversation.java:149)
	at shaded.v9_4.org.eclipse.jetty.client.HttpRequest.abort(HttpRequest.java:794)
	at shaded.v9_4.org.eclipse.jetty.client.HttpDestination.abort(HttpDestination.java:469)
	at shaded.v9_4.org.eclipse.jetty.client.HttpDestination.failed(HttpDestination.java:250)
	at shaded.v9_4.org.eclipse.jetty.client.AbstractConnectionPool$1.failed(AbstractConnectionPool.java:140)
	at shaded.v9_4.org.eclipse.jetty.util.Promise$Wrapper.failed(Promise.java:136)
	at com.databricks.rpc.Jetty9Client$DatabricksHttpDestinationOverHTTP$$anon$2.failed(Jetty9Client.scala:1531)
	at shaded.v9_4.org.eclipse.jetty.util.Promise$Wrapper.failed(Promise.java:136)
	at shaded.v9_4.org.eclipse.jetty.client.HttpClient$1$1.failed(HttpClient.java:602)
	at shaded.v9_4.org.eclipse.jetty.client.AbstractConnectorHttpClientTransport.connectFailed(AbstractConnectorHttpClientTransport.java:138)
	at shaded.v9_4.org.eclipse.jetty.client.AbstractConnectorHttpClientTransport$ClientSelectorManager.connectionFailed(AbstractConnectorHttpClientTransport.java:188)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$Connect.failed(ManagedSelector.java:822)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.processConnect(ManagedSelector.java:254)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.access$1400(ManagedSelector.java:62)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$SelectorProducer.processSelected(ManagedSelector.java:543)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector$SelectorProducer.produce(ManagedSelector.java:401)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.produceTask(EatWhatYouKill.java:360)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.doProduce(EatWhatYouKill.java:184)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.tryProduce(EatWhatYouKill.java:171)
	at shaded.v9_4.org.eclipse.jetty.util.thread.strategy.EatWhatYouKill.run(EatWhatYouKill.java:129)
	at shaded.v9_4.org.eclipse.jetty.util.thread.ReservedThreadExecutor$ReservedThread.run(ReservedThreadExecutor.java:388)
	at com.databricks.rpc.ShadedInstrumentedQueuedThreadPool$$anon$2.run(InstrumentedQueuedThreadPool.scala:131)
	at shaded.v9_4.org.eclipse.jetty.util.thread.QueuedThreadPool.runJob(QueuedThreadPool.java:806)
	at shaded.v9_4.org.eclipse.jetty.util.thread.QueuedThreadPool$Runner.run(QueuedThreadPool.java:938)
	at java.base/java.lang.Thread.run(Thread.java:834)
Caused by: java.net.ConnectException: Connection refused
	at java.base/sun.nio.ch.SocketChannelImpl.checkConnect(Native Method)
	at java.base/sun.nio.ch.SocketChannelImpl.finishConnect(SocketChannelImpl.java:779)
	at shaded.v9_4.org.eclipse.jetty.io.SelectorManager.doFinishConnect(SelectorManager.java:355)
	at shaded.v9_4.org.eclipse.jetty.io.ManagedSelector.processConnect(ManagedSelector.java:232)
	... 12 more